# インポート

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm.notebook import tqdm
from pathlib import Path
import scraping
import pickle
import create_table
import create_prediction_population
%load_ext autoreload

In [2]:
%autoreload

In [3]:
# 一時保存ディレクトリ
TMP_DIR = scraping.DATA_DIR / "tmp"

# 開催日を取得

In [ ]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2008-1", to_="2008-12") #2007年以前はhtmlの書き方が変わるので使えない

  0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
race_id_list = scraping.scrape_race_id_list(
    kaisai_date_list=kaisai_date_list, 
    save_dir=TMP_DIR
    )

  0%|          | 0/108 [00:00<?, ?it/s]

# レースページを取得

In [29]:
# "race_id_list.pickle" からデータを読み込み、race_id_list に代入する
with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [6]:
with open(TMP_DIR / "race_id_list.txt", "r") as f:
    race_id_list = f.read().split("\n")

In [ ]:
len(race_id_list)

10362

In [10]:
html_path_race = scraping.scrape_html_race(
    race_id_list=race_id_list,
    skip=True,
    path=True
    )

  0%|          | 0/3454 [00:00<?, ?it/s]

skipped:200806010101
skipped:200806010102
skipped:200806010103
skipped:200806010104
skipped:200806010105
skipped:200806010106
skipped:200806010107
skipped:200806010108
skipped:200806010109
skipped:200806010110
skipped:200806010111
skipped:200806010112
skipped:200808010101
skipped:200808010102
skipped:200808010103
skipped:200808010104
skipped:200808010105
skipped:200808010106
skipped:200808010107
skipped:200808010108
skipped:200808010109
skipped:200808010110
skipped:200808010111
skipped:200808010112
skipped:200806010201
skipped:200806010202
skipped:200806010203
skipped:200806010204
skipped:200806010205
skipped:200806010206
skipped:200806010207
skipped:200806010208
skipped:200806010209
skipped:200806010210
skipped:200806010211
skipped:200806010212
skipped:200808010201
skipped:200808010202
skipped:200808010203
skipped:200808010204
skipped:200808010205
skipped:200808010206
skipped:200808010207
skipped:200808010208
skipped:200808010209
skipped:200808010210
skipped:200808010211
skipped:20080

In [15]:
len(html_path_race)

3454

In [3]:
# レースのhtmlファイルがなければ取得する
HTML_RACE_DIR = Path("..","data","html","race_test")
html_path_race = list(HTML_RACE_DIR.glob("2024*.bin"))
len(html_path_race)

1113

In [11]:
# レース結果テーブルを作成
results = create_table.table_results(
    html_path_list=html_path_race,
    save_filename="results09-24.csv"
    )

  0%|          | 0/3454 [00:00<?, ?it/s]

Error processing ../data/html/race_test/200808020204.bin
Traceback (most recent call last):
  File "/home/load0/競馬AI/keiba_race_ai/keiba_ai/common/src/create_table.py", line 34, in table_results
    df = pd.read_html(html)[0]
         ^^^^^^^^^^^^^^^^^^
  File "/home/load0/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/html.py", line 1240, in read_html
    return _parse(
           ^^^^^^^
  File "/home/load0/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/html.py", line 1008, in _parse
    df = _data_to_frame(data=table, **kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/load0/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/html.py", line 875, in _data_to_frame
    with TextParser(body, header=header, **kwargs) as tp:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/load0/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 2053, in TextParser
    return TextFileReader(*args, **kwd

In [6]:
# 欠損チェック
results.isnull().sum()

race_id            0
着順                 0
枠番                 0
馬番                 0
馬名                 0
性齢                 0
斤量                 0
騎手                 0
タイム             5233
着差             50120
単勝                 0
人気              2226
馬体重                0
調教師                0
horse_id           0
jockey_id          0
trainer_id         0
owner_id      299669
ﾀｲﾑ指数         633391
通過            633391
上り            633391
調教ﾀｲﾑ         633391
厩舎ｺﾒﾝﾄ        633391
備考            633391
馬主            633391
賞金(万円)        633391
dtype: int64

In [5]:
# 重複チェック
results.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

## レース情報テーブルの作成

In [16]:
race_info = create_table.table_race_info(
    html_path_list=html_path_race,
    save_filename = "race_info09-24.csv"
    )

  0%|          | 0/3454 [00:00<?, ?it/s]

'NoneType' object has no attribute 'find' at 200808020204
'NoneType' object has no attribute 'find' at 200808020804


In [17]:
# 欠損チェック
race_info.isnull().sum()

race_id    0
title      0
info_1     0
info_2     0
dtype: int64

In [18]:
# 重複チェック
race_info.duplicated(subset=["race_id"]).sum()

np.int64(0)

## 払い戻しテーブルの作成

In [ ]:
return_tables = create_table.create_return_tables(
    html_path_list=html_path_race,
    save_filename="return_tables.csv"
    )

  0%|          | 0/3456 [00:00<?, ?it/s]

In [ ]:
# 欠損チェック
return_tables.isnull().sum()

In [ ]:
# 重複チェック
return_tables.duplicated(subset=["race_id"]).sum()

# horseページからの取得

In [12]:
# レース結果テーブルから馬idを取得    
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

11020

In [14]:
# 馬idからホースページの情報を取得
html_path_horse = scraping.scrape_html_horse(
    horse_id_list=horse_id_list,
    skip=True,
    path=True
    )

  0%|          | 0/11020 [00:00<?, ?it/s]

skipped:2005102028
skipped:2005101044
skipped:2005103612
skipped:2005102035
skipped:2005104726
skipped:2005107051
skipped:2005106223
skipped:2005102362
skipped:2005102367
skipped:2005109290
skipped:2005101441
skipped:2005103849
skipped:2005106540
skipped:2005104021
skipped:2005102678
skipped:2005105210
skipped:2005105117
skipped:2005110184
skipped:2005104300
skipped:2005105241
skipped:2005100098
skipped:2005102020
skipped:2005105187
skipped:2005109178
skipped:2005103252
skipped:2005102621
skipped:2005102582
skipped:2005106852
skipped:2005103100
skipped:2005105768
skipped:2005101219
skipped:2005110005
skipped:2005105946
skipped:2005106341
skipped:2005110114
skipped:2005102213
skipped:2005102940
skipped:2005104600
skipped:2005101110
skipped:2004101991
skipped:2004105426
skipped:2004104907
skipped:2003104556
skipped:2003101985
skipped:2004106683
skipped:2003103468
skipped:2003103802
skipped:2004110063
skipped:2003109459
skipped:2002100921
skipped:2003100380
skipped:2003104234
skipped:2004

In [13]:
# horse_idを保存
with open(TMP_DIR / "horse_id_list.txt", "w") as f:
        f.write("\n".join(horse_id_list))

In [15]:
# "horse_id_list.pickle" からデータを読み込み、horse_id_list に代入する
with open(TMP_DIR/ "horse_id_list.txt", "r") as f:
    horse_id_list = f.read().split("\n")

In [3]:
# カーネルを再起動する必要があるとき用
results = pd.read_csv(create_table.RAWDF_DIR / "results12-24.csv", sep="\t")
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

67764

In [ ]:
len(html_path_horse)

20504

In [ ]:
# ホースページのhtmlファイルがなければ取得する
HTML_HORSE_DIR = Path("..","data","html","horse")
html_path_horse = sorted(
    HTML_HORSE_DIR.glob("*.bin"),  # .bin ファイルをリスト化
    key=lambda x: x.stat().st_mtime,  # ファイルの最終更新日時でソート
    reverse=True  # 新しい順（降順）に並べる
)
# len(html_path_horse)

In [39]:
# ホースページのhtmlファイルがなければ取得する
HTML_HORSE_DIR = Path("..","data","html","horse")
html_path_horse = list(HTML_HORSE_DIR.glob("*.bin"))
len(html_path_horse)

40305

In [19]:
# 馬の過去成績テーブルを作製
results_horse = create_table.table_horse_results(
    html_path_list=html_path_horse,
    save_filename="horse_results09-24.csv"
    )

  0%|          | 0/11020 [00:00<?, ?it/s]

In [20]:
# 欠損チェック
results_horse.isnull().sum()

horse_id         0
日付               0
開催               0
天気             544
R              243
レース名             4
映像          317712
頭数              16
枠番             272
馬番               4
オッズ           4404
人気            2634
着順             615
騎手             103
斤量              49
距離              45
馬場             416
馬場指数        131028
タイム           3935
着差            4142
ﾀｲﾑ指数       131067
通過            3639
ペース         107084
上り           40736
馬体重             45
厩舎ｺﾒﾝﾄ      317712
備考          317712
勝ち馬(2着馬)       838
賞金          178406
dtype: int64

In [21]:
# 重複チェック
results_horse.duplicated(subset=["horse_id", "日付"]).sum()

np.int64(0)

# 血統データをスクレイピング

In [15]:
html_path_ped = scraping.scrape_html_ped(
    horse_id_list = horse_id_list,
    skip=True,
    path=True
    )

  0%|          | 0/11020 [00:00<?, ?it/s]

skipped:2005102028
skipped:2005101044
skipped:2005103612
skipped:2005102035
skipped:2005104726
skipped:2005107051
skipped:2005106223
skipped:2005102362
skipped:2005102367
skipped:2005109290
skipped:2005101441
skipped:2005103849
skipped:2005106540
skipped:2005104021
skipped:2005102678
skipped:2005105210
skipped:2005105117
skipped:2005110184
skipped:2005104300
skipped:2005105241
skipped:2005100098
skipped:2005102020
skipped:2005105187
skipped:2005109178
skipped:2005103252
skipped:2005102621
skipped:2005102582
skipped:2005106852
skipped:2005103100
skipped:2005105768
skipped:2005101219
skipped:2005110005
skipped:2005105946
skipped:2005106341
skipped:2005110114
skipped:2005102213
skipped:2005102940
skipped:2005104600
skipped:2005101110
skipped:2004101991
skipped:2004105426
skipped:2004104907
skipped:2003104556
skipped:2003101985
skipped:2004106683
skipped:2003103468
skipped:2003103802
skipped:2004110063
skipped:2003109459
skipped:2002100921
skipped:2003100380
skipped:2003104234
skipped:2004

In [48]:
len(html_path_ped)

20414

In [29]:
# 血統データのhtmlファイルがなければ取得する
HTML_PED_DIR = Path("..","data","html","ped")
html_path_ped = list(HTML_PED_DIR.glob("*.bin"))
len(html_path_ped)

55279

In [22]:
# 血統データのテーブルを作成
ped = create_table.create_peds(
    html_path_list=html_path_ped,
    save_filename="peds09-24.csv"
    )

  0%|          | 0/11020 [00:00<?, ?it/s]

table not found at 2005110033


In [23]:
# 欠損チェック
ped.isnull().sum()

horse_id    0
ped_0       0
ped_1       0
ped_2       0
ped_3       0
           ..
ped_57      0
ped_58      0
ped_59      0
ped_60      0
ped_61      0
Length: 63, dtype: int64

In [24]:
# 重複チェック
ped.duplicated(subset=["horse_id"]).sum()

np.int64(0)

# 各種リーディングテーブルの取得

In [5]:
# 各種リーディングのデータを取得
for year in [2007]:
    scraping.scrape_html_leading(
        leading_type="jockey_leading",
        year=year,
        pages = [1,2,3,4,5],
        skip= False,
    )

  0%|          | 0/5 [00:00<?, ?it/s]

## 騎手リーディングテーブルの作成

In [6]:
html_path_jockey_leading = list(
    (scraping.HTML_LEADING_DIR/ "jockey_leading").glob("*.bin")
)
html_path_jockey_leading, len(html_path_jockey_leading)

([PosixPath('../data/html/leading/jockey_leading/2021_02.bin'),
  PosixPath('../data/html/leading/jockey_leading/2011_05.bin'),
  PosixPath('../data/html/leading/jockey_leading/2020_02.bin'),
  PosixPath('../data/html/leading/jockey_leading/2013_04.bin'),
  PosixPath('../data/html/leading/jockey_leading/2018_02.bin'),
  PosixPath('../data/html/leading/jockey_leading/2023_02.bin'),
  PosixPath('../data/html/leading/jockey_leading/2008_04.bin'),
  PosixPath('../data/html/leading/jockey_leading/2013_02.bin'),
  PosixPath('../data/html/leading/jockey_leading/2012_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2017_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2022_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2013_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2009_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2015_03.bin'),
  PosixPath('../data/html/leading/jockey_leading/2021_01.bin'),
  PosixPath('../data/html/leading/jockey

In [7]:
# 騎手リーディングのテーブルを作成j
jockey_leading = create_table.create_jockey_leading(
    html_path_list=html_path_jockey_leading,
    save_filename="jockey_leading.csv"
    )

  0%|          | 0/79 [00:00<?, ?it/s]

In [8]:
# page_idの年を取得して、jockey_leadingに追加、年ごとに集計する
jockey_leading["year"] = jockey_leading["page_id"].apply(lambda x: int(x.split("_")[0]))
jockey_leading["year"].value_counts().sort_index()

year
2007    247
2008    247
2009    242
2010    248
2011    223
2012    154
2013    180
2014     94
2015    184
2016    137
2017    153
2018    147
2019    144
2020    185
2021    155
2022    196
2023    198
2024    200
Name: count, dtype: int64

In [9]:
# 欠損チェック
jockey_leading.isnull().sum()

page_id      0
jockey_id    0
順位           0
騎手名          1
所属           0
生年月日         0
1着           0
2着           0
3着           0
着外           0
重賞_出走        0
重賞_勝利        0
特別_出走        0
特別_勝利        0
平場_出走        0
平場_勝利        0
芝_出走         0
芝_勝利         0
ダート_出走       0
ダート_勝利       0
勝率           0
連対率          0
複勝率          0
収得賞金(万円)     0
代表馬          1
year         0
dtype: int64

In [44]:
# 重複チェック
jockey_leading.duplicated(subset=["jockey_id", "page_id"]).sum()

np.int64(0)

## 調教師リーディングの作成

In [10]:
html_path_trainer_leading = list(
    (scraping.HTML_LEADING_DIR/ "trainer_leading").glob("*bin")
)

In [11]:
html_path_trainer_leading, len(html_path_trainer_leading)

([PosixPath('../data/html/leading/trainer_leading/2021_02.bin'),
  PosixPath('../data/html/leading/trainer_leading/2011_05.bin'),
  PosixPath('../data/html/leading/trainer_leading/2007_07.bin'),
  PosixPath('../data/html/leading/trainer_leading/2020_02.bin'),
  PosixPath('../data/html/leading/trainer_leading/2013_04.bin'),
  PosixPath('../data/html/leading/trainer_leading/2019_05.bin'),
  PosixPath('../data/html/leading/trainer_leading/2022_05.bin'),
  PosixPath('../data/html/leading/trainer_leading/2013_06.bin'),
  PosixPath('../data/html/leading/trainer_leading/2018_02.bin'),
  PosixPath('../data/html/leading/trainer_leading/2023_02.bin'),
  PosixPath('../data/html/leading/trainer_leading/2008_04.bin'),
  PosixPath('../data/html/leading/trainer_leading/2013_02.bin'),
  PosixPath('../data/html/leading/trainer_leading/2012_03.bin'),
  PosixPath('../data/html/leading/trainer_leading/2017_03.bin'),
  PosixPath('../data/html/leading/trainer_leading/2022_03.bin'),
  PosixPath('../data/html

In [12]:
trainer_leading = create_table.create_trainer_leading(
    html_path_list=html_path_trainer_leading,
    save_filename="trainer_leading.csv"
    )

  0%|          | 0/93 [00:00<?, ?it/s]

In [13]:
# 重複チェック
trainer_leading.duplicated(subset=["trainer_id", "page_id"]).sum()

np.int64(0)

In [14]:
# page_idの年を取得して、trainer_leadingに追加、年ごとに集計する
trainer_leading["year"] = trainer_leading["page_id"].apply(lambda x: int(x.split("_")[0]))
trainer_leading["year"].value_counts().sort_index()

year
2007    330
2008    283
2009    319
2010    275
2011    308
2012    231
2013    252
2014    250
2017    250
2018    152
2019    231
2020    229
2021    227
2022    230
2023    234
2024    227
Name: count, dtype: int64

## 種牡馬リーディングの作成

In [15]:
html_path_sire_leading = list(
    (scraping.HTML_LEADING_DIR/ "sire_leading").glob("*.bin")
)

In [17]:
sire_leading = create_table.create_sire_leading(
    html_path_list=html_path_sire_leading,
    save_filename="sire_leading.csv"
    )

  0%|          | 0/143 [00:00<?, ?it/s]

## BMSリーディングの作成（BMSとは母方の祖父）

In [18]:
html_path_bms_leading = list(
    (scraping.HTML_LEADING_DIR/ "bms_leading").glob("*.bin")
)

In [19]:
# 実行しなくても学習は回せる
bms_leading = create_table.create_bms_leading(
    html_path_list=html_path_bms_leading,
    save_filename="bms_leading.csv"
    )

  0%|          | 0/143 [00:00<?, ?it/s]